In [2]:
from dotenv import load_dotenv
import github as gh

In [3]:
load_dotenv()
GITHUB_ACCESS_TOKEN=os.getenv('GITHUB_ACCESS_TOKEN')
g=gh.Github(GITHUB_ACCESS_TOKEN)

In [4]:
print(g.get_user())

AuthenticatedUser(login=None)


In [14]:
query='tensorflow language:python created:2021-04-01..2021-05-01'
result=g.search_repositories(query)

In [15]:
print(result.totalCount)

503


In [7]:
print(dir(result))

['_PaginatedListBase__elements', '_PaginatedListBase__fetchToIndex', '_PaginatedList__contentClass', '_PaginatedList__firstParams', '_PaginatedList__firstUrl', '_PaginatedList__headers', '_PaginatedList__list_item', '_PaginatedList__nextParams', '_PaginatedList__nextUrl', '_PaginatedList__parseLinkHeader', '_PaginatedList__requester', '_PaginatedList__reverse', '_PaginatedList__totalCount', '_Slice', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_couldGrow', '_fetchNextPage', '_getLastPageUrl', '_grow', '_isBiggerThan', '_reversed', 'get_page', 'reversed', 'totalCount']


In [16]:
for repository in result:
    print(repository.git_url)
    break

git://github.com/LynnHo/EigenGAN-Tensorflow.git
